In [1]:
import re
import requests
from bs4 import BeautifulSoup
from urllib import urlopen
import urllib2
import pynlpir
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import requests 
from StringIO import StringIO
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 
import datetime as dt # module for manipulating dates and times
import numpy.linalg as lin # module for performing linear algebra operations}
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from pandas import Series
from pandas import DataFrame
import timeit
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import timeit
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
pynlpir.open()
pynlpir.open(encoding='utf-8')

## 先做一个根据 小编推荐算 相似度的 implementation

**step1: 拿到 general 小编推荐的 tag list**

In [3]:
user_app_tag_table_final = pd.read_csv("D:/R_data/job_for_XYZS/user_app_tag_table_final_overall",sep=",",encoding = "utf-8")

In [4]:
user_app_tag_table_final = user_app_tag_table_final.drop(user_app_tag_table_final.columns[0],axis = 1)

In [5]:
xiao_bian_tui_jian_keyword_table = pd.DataFrame(user_app_tag_table_final["xiao_bian_tui_jian_keyword"])

In [6]:
xiao_bian_tui_jian_keyword_table = xiao_bian_tui_jian_keyword_table.dropna()

In [7]:
xiao_bian_tui_jian_keyword = xiao_bian_tui_jian_keyword_table.xiao_bian_tui_jian_keyword

In [8]:
overall_keyword_list = []

In [9]:
for a in xiao_bian_tui_jian_keyword:
    if a == ",":
        continue
    else:
        overall_keyword_list.append(a)

In [10]:
overall_keyword_content = " ".join(overall_keyword_list)

In [11]:
from collections import Counter

In [12]:
keyword_list_single = overall_keyword_content.split(" ")

In [13]:
counts = Counter(keyword_list_single)

In [14]:
keyword_series = pd.Series(counts)

In [15]:
xiao_bian_tui_jian_keyword_count_table = pd.DataFrame(keyword_series)

In [16]:
xiao_bian_tui_jian_keyword_count_table=xiao_bian_tui_jian_keyword_count_table.reset_index()

In [17]:
xiao_bian_tui_jian_keyword_count_table = xiao_bian_tui_jian_keyword_count_table.rename(columns={"index":"keyword_of_xiao_bian_tui_jian",0:"count"})

In [18]:
xiao_bian_tui_jian_keyword_count_table["track_id"] = xiao_bian_tui_jian_keyword_count_table.index

In [19]:
xiao_bian_tui_jian_keyword_count_table

,keyword_of_xiao_bian_tui_jian,count,track_id
0,10W,1,0
1,2G3G,2,1
2,3D,1,2
3,3DMMO,2,3
4,5V5,3,4
5,APP,14,5
6,APP.,1,6
7,ARPG,8,7
8,Air,1,8
9,AirPort,1,9


step 2: 把app 的表拿出来

In [20]:
url_app_table_overall = pd.read_csv("D:/R_data/job_for_XYZS/user_tag_list_scrapping/dt/craw_info_table_overall_overall_with_keyword_outcome",sep=",",
                            encoding= "utf-8")

In [21]:
url_app_table_overall = url_app_table_overall.drop(url_app_table_overall.columns[0],axis = 1)

In [22]:
url_app_table_overall = url_app_table_overall.fillna("void")

**经过预处理之后 小编推荐一栏 没有keyword的只有这三个了**

In [23]:
url_app_table_overall[url_app_table_overall.xiao_bian_tui_jian_keyword == "void"]

,app_jian_jie,app_name,banben_jian_jie,download_time,page_url,rank,rating,sub_cate,update_date,version,view_count,xiao_bian_tui_jian,app_name_keyword,app_name_keyword_score,xiao_bian_tui_jian_keyword,xiao_bian_tui_jian_keyword_score
762,,免费小说阅读-快看免费小说掌,版本3.5.0中的新功能1.修复一...,5640,http://www.xyzs.com/app/861774044.html,99,void,图书,更新：2016-06-10,版本：3.5.0,1,void,免费 小说 阅读,2.69 2.69 2.0,void,void
804,,"火球理财-10%高收益活期理财,一",版本2.3.2中的新功能1.新增新...,4554,http://www.xyzs.com/app/913181869.html,141,void,财务,更新：2016-08-20,版本：2.3.2,1,void,理财 火球 收益,4.39 2.0 2.0,void,void
880,,方块消消乐-爱消除的六边形俄,版本1.0.2中的新功能新的分数分...,1.0万,http://www.xyzs.com/app/1057169289.html,223,void,智力游戏,更新：2015-12-06,版本：1.0.2,1,void,方块 消除 六边形 消,2.0 2.0 2.0 1.46,void,void


**step 3 拉matrix**

In [24]:
tui_jian_Ctoken_table = pd.DataFrame(columns = xiao_bian_tui_jian_keyword_count_table.keyword_of_xiao_bian_tui_jian.values,index = url_app_table_overall.index)

In [25]:
tui_jian_Ctoken_table = tui_jian_Ctoken_table.fillna(0.0)

In [26]:
tui_jian_Ctoken_table

,10W,2G3G,3D,3DMMO,5V5,APP,APP.,ARPG,Air,AirPort,...,鸭子,鹰,麻将,黄帝内经,黄磊,鼓舞,鼻梁,鼻涕,龙,龙洞
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


step4: 把每个app 放进去 并计算keyword 出现次数

In [27]:
for a in tui_jian_Ctoken_table.index:
    xiao_bian_tui_jian_keyword_str = url_app_table_overall.ix[a,["xiao_bian_tui_jian_keyword"]].values[0]
    xiao_bian_tui_jian_keyword_list = xiao_bian_tui_jian_keyword_str.split(" ")
    for keyword in xiao_bian_tui_jian_keyword_list:
        if keyword not in list(xiao_bian_tui_jian_keyword_count_table.keyword_of_xiao_bian_tui_jian):
            continue
        
        else:
            keyword_index_in_list = list(xiao_bian_tui_jian_keyword_count_table.keyword_of_xiao_bian_tui_jian).index(keyword)    
            tui_jian_Ctoken_table.ix[a,tui_jian_Ctoken_table.columns[keyword_index_in_list]]+=1

In [28]:
tui_jian_Ctoken_table

,10W,2G3G,3D,3DMMO,5V5,APP,APP.,ARPG,Air,AirPort,...,鸭子,鹰,麻将,黄帝内经,黄磊,鼓舞,鼻梁,鼻涕,龙,龙洞
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**现在 我们也有了算好出现次数的 所有app 的matrix了 下面要做的就是 加入一个用户的小编推荐tag list 然后再给他推荐 app**

下面 先计算用户tag list 出现的次数 

In [29]:
user_app_tag_table_final = pd.read_csv("D:/R_data/job_for_XYZS/user_app_tag_table_final_overall",sep=",",encoding = "utf-8")

In [30]:
user_app_tag_table_final = user_app_tag_table_final.drop(user_app_tag_table_final.columns[0],axis = 1)

In [31]:
user_app_tag_table_final

,overall_app_title,overall_xiao_bian_tui_jian,user_fingerprint,overall_xiao_bian_tui_jian_pre,app_name_keyword,xiao_bian_tui_jian_keyword,app_name_keyword_score,xiao_bian_tui_jian_keyword_score
0,"Myst FREE,时空猎人-高清版,秒拍,橙光游戏中心,SlickPic: Upload ...","void, 体验格斗快感，完美再现超豪爽打击快感！ , 最潮最美拍摄体验,女生必备自拍神器!...",1719313972,", 体验格斗快感，完美再现超豪爽打击快感！ , 最潮最美拍摄体验,女生必备自拍神器! , ...",游戏 黄晓 免费版 免费 壁纸 英语 亚琛 Zoom 大师 动物 卡车 汽车 怪兽 孩子 口...,可以 应用 游戏 体验 提供 视频 宫斗 糖果王国 糖果 生活 平台 官方 拍摄 卡车 编辑...,9.84 6.6 6.46 6.43 6.2 6.08 5.5 5.26 4.74 4.74...,10.81 9.22 8.56 8.02 7.94 7.77 6.6 6.46 6.43 6...
1,"Coloring Zoo: Finger Painting FREE,字拍,杀手鲨鱼袭击模拟...","void, 是一款专注于把中文添加到图片上的App , 吃掉眼前所有的人类! , 提供给JB...",1481736560,", 是一款专注于把中文添加到图片上的App , 吃掉眼前所有的人类! , 提供给JBL和Y...",Coloring Finger Painting FREE WindGuru 微信 musi...,设备 翻译 进化 Yurbuds 微信 侏罗纪 专注 中文 添加 图片 人类 提供 品牌 相...,2.4 2.4 2.4 2.4 2.4 2.4 2.4 2.4 2.4 2.4 2.4 2....,4.39 4.39 3.1 2.4 2.4 2.2 2.0 2.0 2.0 2.0 2.0 ...
2,"龙珠直播,携程旅行-订机票酒店,唯品会,战争兵团 － 起源,鳄鱼射击3D,免费小说阅读-快看...","为您实时提供全球最全最热门的赛事直播,你想要的都在这里! , #爱过年,攒福利# 人人有8...",35840995,"为您实时提供全球最全最热门的赛事直播,你想要的都在这里! , #爱过年,攒福利# 人人有8...",免费小说 魔兽 小说 直播 免费 战斗机 蜀山 旅行 机票 酒店 战争 兵团 起源 鳄鱼 射...,战斗 人人 杀死 鳄鱼 阅读 创造 享受 全球 直播 MMORPG 蜀山 提供 赛事 过年 ...,6.88 6.46 3.1 2.64 2.64 2.35 2.2 2.0 2.0 2.0 2...,6.28 4.39 4.39 4.39 4.39 4.39 4.39 4.27 4.27 2...
3,"橙光游戏中心,BOXFiSH-盒子鱼英语,AMS WIFICAM,橙光游戏中心,橙光游戏中心...","有爱有梦有趣 有基有腐有CP 帅哥美女如云 穿越宫斗成瘾 , 不用再面对呆板枯燥的英语作业...",2740574975,"有爱有梦有趣 有基有腐有CP 帅哥美女如云 穿越宫斗成瘾 , 不用再面对呆板枯燥的英语作业...",浙江 移动 BOXFiSH WIFICAM Zalo 中心 游戏 盒子 英语 全民 传说 吉...,宫斗 视频 棋牌 精选 软件 有基 仙侠 3DMMO 浙江 美女 穿越 面对 英语 作业 学...,2.96 2.69 2.4 2.4 2.4 2.18 2.0 2.0 2.0 2.0 2.0...,6.6 4.39 4.39 4.39 2.69 2.4 2.4 2.4 2.2 2.0 2....
4,"彩吧,生意街,e片红,金太阳,E动生命,异度支付,移动手机贷,招钱宝,荷包,通达信,天天记账...","综合性的彩票资讯平台,精准预测彩票走势,极速推送实时开奖信息 , 先行赔付创业保障计划,让...",3038159794,"综合性的彩票资讯平台,精准预测彩票走势,极速推送实时开奖信息 , 先行赔付创业保障计划,让...",手机 支付 记账 中国 生意 太阳 生命 移动 荷包 通达 大学 集邮 宝,移动 提供 客户端 服务 资讯平台 用户 证券 管理 记账 客户 资讯 推出 金融 软件 消...,4.39 2.69 2.69 2.2 2.0 2.0 2.0 2.0 2.0 2.0 2.0...,7.94 7.77 6.88 6.66 6.46 6.43 6.2 6.2 6.2 4.95...
5,"Junos Pulse,橙光游戏中心,EasyConnect,视频监控,爱奇艺视频,Play...","void, 有爱有梦有趣 有基有腐有CP 帅哥美女如云 穿越宫斗成瘾 ,void, 内部管理...",183884223,", 有爱有梦有趣 有基有腐有CP 帅哥美女如云 穿越宫斗成瘾 , , 内部管理,安全防护,...",视频 游戏 Junos Pulse EasyConnect 爱奇艺 PlayKids Tea...,宫斗 黄磊 游戏 应用 有基 钱江晚报 瞬间 浙江 安全 美女 穿越 管理 防护 监控 师徒...,4.39 4.12 2.4 2.4 2.4 2.4 2.4 2.4 2.4 2.4 2.4 ...,6.6 5.5 4.39 4.39 2.4 2.4 2.2 2.2 2.19 2.0 2.0...
6,"天籁,爱上煲仔饭,起烘,吃吧,眼萌,特色小吃,按一按,外卖库,安居宝APP,吃货帮,食-友,...","包含来自瑞士的罗春湖畔,玫瑰山麓和阿尔卑斯山的风声,流水声,虫声,鸟鸣, 浪涛和风啸等天然...",4048061964,"包含来自瑞士的罗春湖畔,玫瑰山麓和阿尔卑斯山的风声,流水声,虫声,鸟鸣, 浪涛和风啸等天然...",天籁 爱上 特色 小吃 安居 配色 美食 婚礼 库 APP,美食 烘焙 专业 功能 学会 美味 生活 必备 婚礼 APP. 瑞士 包含 来自 湖畔 玫瑰...,2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 1.46 1.2,6.56 6.2 6.2 4.39 4.39 4.39 4.39 4.39 4.39 2.4...
7,"妙队,吃货娃娃：Coco的瘦身之战,优买,Ghaye,Nice Weather,石头理财,即...","让您适时了解餐厅的排队情况,直接从手机远程取号排队 ,void, 让移动生活更美好! , ...",1136107332,"让您适时了解餐厅的排队情况,直接从手机远程取号排队 , , 让移动生活更美好! , 格雅语...",全球 天气 建筑 Coco Ghaye Nice Weather DiffWeather 娃...,生活 排队 手机 语言 宝宝 瞬间 了解 餐厅 情况 移动 培训 中心 推出 产品 学习 客...,4.39 4.39 4.39 2.4 2.4 2.4 2.4 2.4 2.0 2.0 2.0...,7.77 4.39 4.39 4.39 4.39 2.2 2.0 2.0 2.0 2.0 2...
8,"NVSIP,作业帮-学霸搜题,Vi?t Tivi HD - Xem K+,K+NS,K+PC...","void, 作业一拍,秒出答案! ,void, 一款火爆的美女视频直播,视频交友软件 , 有...",103880027,", 作业一拍,秒出答案! , , 一款火爆的美女视频直播,视频交友软件 , 有爱有梦有趣 ...",NVSIP Tivi K+NS K+PC BoBo BOXFiSH 中心 作业 美女 直播 ...,宫斗 视频 作业 美女 有基 答案 直播 交友 软件 穿越 市场 飙升 金花 游戏 土豪 错...,2.4 2.4 2.4 2.4 2.4 2.4 2.18 2.0 2.0 2.0 2.0 2...,6.6 4.39 4.27 3.9 2.4 2.0 2.0 2.0 2.0 2.0 2.0 ...
9,"TouristEraser (旅游橡皮擦),从高楼跳下的火柴人,阿里钱盾,布卢小子2 Blo...","void,void, 阿里巴巴移动安全部官方出品的一款免费的手机管理应用. , 喜欢这款游戏...",4092804660,", , 阿里巴巴移动安全部官方出品的一款免费的手机管理应用. , 喜欢这款游戏的玩家赶紧下...",邦戈 Pet TouristEraser Bloo Adventure Jungle Bla...,游戏 收集 宠物 中国 玩 阿里巴巴 移动 官方 出品 免费 手机 管理 应用 喜欢 下载 ...,6.6 2.63 2.4 2.4 2.4 2.4 2.4 2.4 2.2 2.0 2.0 2...,6.2 4.39 4.39 2.2 2.07 2.0 2.0 2.0 2.0 2.0 2.0...


**先选 index 是 17 的用户来看一下**

In [32]:
print user_app_tag_table_final.ix[17].overall_app_title

美柚大姨妈助手,宜人贷借款,天天果园,导航犬,一号专车,携程旅行-订机票酒店,同程旅游-错峰出游,百度导航


感觉这个人 比较喜欢出去

In [33]:
user17 = pd.DataFrame(user_app_tag_table_final.ix[17])

In [34]:
user17 = user17.transpose()

In [35]:
user17

,overall_app_title,overall_xiao_bian_tui_jian,user_fingerprint,overall_xiao_bian_tui_jian_pre,app_name_keyword,xiao_bian_tui_jian_keyword,app_name_keyword_score,xiao_bian_tui_jian_keyword_score
17,"美柚大姨妈助手,宜人贷借款,天天果园,导航犬,一号专车,携程旅行-订机票酒店,同程旅游-错峰...","女生经期记录软件，超过1亿女生都在用！ , 最新免费快速服务流程,全网利率最低. ,voi...",2898542516,"女生经期记录软件，超过1亿女生都在用！ , 最新免费快速服务流程,全网利率最低. , , ...",导航 大姨 助手 借款 果园 专车 旅行 机票 酒店 旅游 出游 百度 程 宜人,女生 节后 记录 软件 超过 免费 服务 流程 利率 聚焦 实时 路况 缓解 交通 压力 聚...,3.04 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2...,4.39 2.2 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2.0 2...


In [36]:
user_xiao_bian_tui_jian_keyword_list = user17.xiao_bian_tui_jian_keyword.values[0]

In [37]:
user_xiao_bian_tui_jian_keyword_list = user_xiao_bian_tui_jian_keyword_list.split(" ")

In [38]:
user_Ctoken_matrix = pd.DataFrame(columns =tui_jian_Ctoken_table.columns,index=[1344])

In [39]:
user_Ctoken_matrix = user_Ctoken_matrix.fillna(0.0)

In [40]:
user_Ctoken_matrix

,10W,2G3G,3D,3DMMO,5V5,APP,APP.,ARPG,Air,AirPort,...,鸭子,鹰,麻将,黄帝内经,黄磊,鼓舞,鼻梁,鼻涕,龙,龙洞
1344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
for d in user_Ctoken_matrix.index:
    xiao_bian_tui_jian_user = user17.ix[17,["xiao_bian_tui_jian_keyword"]].values[0]
    xiao_bian_tui_jian_user_list = xiao_bian_tui_jian_user.split(" ")
    for keyword in xiao_bian_tui_jian_user_list:
        if keyword not in list(xiao_bian_tui_jian_keyword_count_table.keyword_of_xiao_bian_tui_jian):
            continue
        
        else:
            keyword_index_in_list = list(xiao_bian_tui_jian_keyword_count_table.keyword_of_xiao_bian_tui_jian).index(keyword)    
            user_Ctoken_matrix.ix[d,user_Ctoken_matrix.columns[keyword_index_in_list]]+=1        

In [42]:
user_Ctoken_matrix

,10W,2G3G,3D,3DMMO,5V5,APP,APP.,ARPG,Air,AirPort,...,鸭子,鹰,麻将,黄帝内经,黄磊,鼓舞,鼻梁,鼻涕,龙,龙洞
1344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
user_Ctoken_matrix_test = user_Ctoken_matrix.transpose()

In [45]:
user_Ctoken_matrix_test = user_Ctoken_matrix_test.rename(columns={1344:"score"})

In [51]:
user_Ctoken_matrix_test[user_Ctoken_matrix_test.score != 0]

,score
交通,1.0
人人,1.0
免费,1.0
出行,1.0
分享,1.0
利率,1.0
压力,1.0
女友,1.0
女生,1.0
实时,1.0


**下面把app 和 用户 tag matrix 两个合起来**

In [52]:
user_app_Ctoken_occur_table = pd.concat([tui_jian_Ctoken_table,user_Ctoken_matrix],axis = 0)

In [53]:
user_app_Ctoken_occur_table

,10W,2G3G,3D,3DMMO,5V5,APP,APP.,ARPG,Air,AirPort,...,鸭子,鹰,麻将,黄帝内经,黄磊,鼓舞,鼻梁,鼻涕,龙,龙洞
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**下面一步 计算 cos similarity**

In [54]:
user_app_Ctoken_occur_table.as_matrix()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [58]:
dist_1 = cosine_similarity(user_app_Ctoken_occur_table.as_matrix())

In [59]:
sim_matrix_table = pd.DataFrame(dist_1)

In [60]:
sim_matrix_table

,0,1,2,3,4,5,6,7,8,9,...,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344
0,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,1.000000,0.0,0.083333,0.102062,0.000000,0.000000,0.0,0.083333,0.0,...,0.000000,0.000000,0.000000,0.000000,0.144338,0.000000,0.000000,0.000000,0.193649,0.000000
2,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.083333,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129099,0.000000
4,0.000000,0.102062,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.158114,0.000000
5,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,...,0.144338,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061546
6,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.083333,0.0,0.166667,0.000000,0.000000,0.000000,0.0,1.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
user_row = sim_matrix_table.ix[1344]

In [64]:
user_RS_table = pd.DataFrame(user_row)

In [66]:
user_RS_table = user_RS_table.drop(user_RS_table.index[1344],axis = 0)

In [68]:
user_RS_table = user_RS_table.rename(columns={1344:"relevence_score"})

In [70]:
user_RS_table["app_name"] = "void"

In [77]:
for e in user_RS_table.index:
    app_name = url_app_table_overall.ix[e,["app_name"]].values[0]
    user_RS_table.ix[e,["app_name"]] = app_name

In [75]:
print app_name

手机管家


In [79]:
user_RS_table = user_RS_table.sort_values(by=["relevence_score"],ascending = False)

In [93]:
user_RS_table.head()

,relevence_score,app_name
456,0.426401,宜人贷借款
153,0.426401,导航犬
275,0.389249,携程旅行-订机票酒店
197,0.348155,百度导航
451,0.348155,一号专车


In [99]:
user17_app_name_list = user17.overall_app_title.values[0].split(",")

In [100]:
user17_app_name_list

[u'\u7f8e\u67da\u5927\u59e8\u5988\u52a9\u624b',
 u'\u5b9c\u4eba\u8d37\u501f\u6b3e',
 u'\u5929\u5929\u679c\u56ed',
 u'\u5bfc\u822a\u72ac',
 u'\u4e00\u53f7\u4e13\u8f66',
 u'\u643a\u7a0b\u65c5\u884c-\u8ba2\u673a\u7968\u9152\u5e97',
 u'\u540c\u7a0b\u65c5\u6e38-\u9519\u5cf0\u51fa\u6e38',
 u'\u767e\u5ea6\u5bfc\u822a']

In [106]:
user_RS_table["occur_in_viewed_list_tag"]="void"

In [107]:
for a in user_RS_table.index:
    app_name = user_RS_table.ix[a,["app_name"]].values[0]
    if app_name in user17_app_name_list:
        user_RS_table.ix[a,["occur_in_viewed_list_tag"]] = "Yes"
    else:
        user_RS_table.ix[a,["occur_in_viewed_list_tag"]] = "No"

In [110]:
user_RS_table[user_RS_table.occur_in_viewed_list_tag == "No"].head(20)

,relevence_score,app_name,occur_in_viewed_list_tag
457,0.246183,行者,No
1136,0.213201,紫微斗数专业版,No
1295,0.197386,路况通,No
1158,0.174078,中国药品大全,No
898,0.174078,XPlanner 项目管理 MPP 甘特图,No
1035,0.174078,叮当巴士,No
188,0.174078,秒拍,No
1271,0.174078,QQ达人,No
715,0.165145,舞蹈圈,No
888,0.165145,爱共享,No


In [113]:
print user17.overall_app_title.values[0]

美柚大姨妈助手,宜人贷借款,天天果园,导航犬,一号专车,携程旅行-订机票酒店,同程旅游-错峰出游,百度导航


In [118]:
print url_app_table_overall.ix[457].xiao_bian_tui_jian

 最好用的骑行软件,骑行必备 


In [119]:
print url_app_table_overall.ix[1136].xiao_bian_tui_jian

 人人必备查看运势运程的紫微星座软件 


In [121]:
print url_app_table_overall.ix[1295].xiao_bian_tui_jian_keyword

路况 路况简图 微博 交通 全国 城市 @路况通 实时路况 高速路况 路况信息 立体播报 前方道路 周边路况 播报 语音 定制 信息 简图 实时 道路 包含 拥堵 事件 出行 地图 查看 分享 应用 热点 区域 周边 FM103.9 IOS6 Siri Store 五一 北京 内容 提要 长假 出游 必备 导航 助手 无需 查询 位置 行驶 交汇 方向


In [122]:
print url_app_table_overall.ix[1158].xiao_bian_tui_jian

 一款完全免费的掌上医疗软件 


In [125]:
print url_app_table_overall.ix[1035].xiao_bian_tui_jian

  上下班通勤及团体出行定车软件 


In [124]:
user_RS_table[user_RS_table.occur_in_viewed_list_tag == "Yes"]

,relevence_score,app_name,occur_in_viewed_list_tag
456,0.426401,宜人贷借款,Yes
153,0.426401,导航犬,Yes
275,0.389249,携程旅行-订机票酒店,Yes
197,0.348155,百度导航,Yes
451,0.348155,一号专车,Yes
1165,0.348155,美柚大姨妈助手,Yes
1267,0.348155,同程旅游-错峰出游,Yes
718,0.065795,天天果园,Yes
